In [3]:
import pandas as pd
import os

In [4]:
data = pd.read_csv(os.path.join('data', 'abcnews-date-text.csv'), error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [5]:
print(len(documents))
print(documents[:5])

1103665
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [13]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/amaity2/nltk_models...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [15]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)

print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [16]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [17]:
dictionary = gensim.corpora.Dictionary(processed_docs)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [18]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [19]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (483, 1), (4014, 1)]

In [21]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 483 ("rain") appears 1 time.
Word 4014 ("dampen") appears 1 time.


In [22]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5892908644709983),
 (1, 0.38929657403503015),
 (2, 0.4964985198530063),
 (3, 0.5046520328695662)]


In [23]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [24]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.029*"elect" + 0.019*"say" + 0.018*"death" + 0.017*"hospit" + 0.016*"tasmanian" + 0.015*"labor" + 0.013*"deal" + 0.013*"china" + 0.011*"polit" + 0.011*"talk"
Topic: 1 
Words: 0.020*"nation" + 0.018*"coast" + 0.016*"help" + 0.016*"countri" + 0.015*"state" + 0.015*"chang" + 0.015*"health" + 0.013*"hour" + 0.013*"indigen" + 0.012*"water"
Topic: 2 
Words: 0.019*"canberra" + 0.018*"market" + 0.014*"rise" + 0.014*"west" + 0.014*"australian" + 0.014*"turnbul" + 0.013*"price" + 0.013*"share" + 0.012*"victoria" + 0.011*"bank"
Topic: 3 
Words: 0.062*"polic" + 0.023*"crash" + 0.019*"interview" + 0.018*"miss" + 0.018*"shoot" + 0.015*"investig" + 0.013*"arrest" + 0.013*"driver" + 0.011*"search" + 0.011*"offic"
Topic: 4 
Words: 0.029*"charg" + 0.027*"court" + 0.021*"murder" + 0.018*"woman" + 0.018*"face" + 0.016*"alleg" + 0.015*"brisban" + 0.015*"live" + 0.015*"jail" + 0.014*"perth"
Topic: 5 
Words: 0.035*"australia" + 0.021*"melbourn" + 0.021*"world" + 0.017*"open" + 0.014*"final"

In [25]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.022*"trump" + 0.007*"novemb" + 0.007*"monday" + 0.006*"wednesday" + 0.005*"decemb" + 0.005*"leagu" + 0.005*"open" + 0.005*"world" + 0.004*"final" + 0.004*"round"
Topic: 1 Word: 0.016*"polic" + 0.014*"interview" + 0.013*"charg" + 0.013*"crash" + 0.013*"murder" + 0.012*"woman" + 0.010*"death" + 0.009*"court" + 0.008*"shoot" + 0.008*"die"
Topic: 2 Word: 0.010*"weather" + 0.008*"queensland" + 0.008*"farm" + 0.007*"flood" + 0.006*"friday" + 0.006*"farmer" + 0.006*"damag" + 0.006*"rain" + 0.006*"dairi" + 0.006*"cyclon"
Topic: 3 Word: 0.009*"drug" + 0.008*"abbott" + 0.008*"plead" + 0.007*"guilti" + 0.006*"polic" + 0.006*"charg" + 0.005*"court" + 0.005*"disabl" + 0.005*"know" + 0.005*"spring"
Topic: 4 Word: 0.007*"hobart" + 0.006*"andrew" + 0.006*"jam" + 0.006*"april" + 0.005*"scott" + 0.005*"zealand" + 0.005*"flight" + 0.004*"breakfast" + 0.004*"paul" + 0.004*"drone"
Topic: 5 Word: 0.017*"countri" + 0.017*"hour" + 0.014*"rural" + 0.013*"market" + 0.011*"news" + 0.008*"share" 

In [26]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [27]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.41999998688697815	 
Topic: 0.026*"south" + 0.025*"kill" + 0.015*"island" + 0.013*"fall" + 0.011*"attack" + 0.009*"forc" + 0.009*"christma" + 0.009*"shark" + 0.008*"east" + 0.007*"northern"

Score: 0.2199999988079071	 
Topic: 0.020*"nation" + 0.018*"coast" + 0.016*"help" + 0.016*"countri" + 0.015*"state" + 0.015*"chang" + 0.015*"health" + 0.013*"hour" + 0.013*"indigen" + 0.012*"water"

Score: 0.2199999988079071	 
Topic: 0.035*"australia" + 0.021*"melbourn" + 0.021*"world" + 0.017*"open" + 0.014*"final" + 0.013*"donald" + 0.011*"sydney" + 0.010*"leagu" + 0.010*"take" + 0.010*"win"

Score: 0.019999999552965164	 
Topic: 0.029*"elect" + 0.019*"say" + 0.018*"death" + 0.017*"hospit" + 0.016*"tasmanian" + 0.015*"labor" + 0.013*"deal" + 0.013*"china" + 0.011*"polit" + 0.011*"talk"

Score: 0.019999999552965164	 
Topic: 0.019*"canberra" + 0.018*"market" + 0.014*"rise" + 0.014*"west" + 0.014*"australian" + 0.014*"turnbul" + 0.013*"price" + 0.013*"share" + 0.012*"victoria" + 0.011*"bank"


In [28]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5350757241249084	 
Topic: 0.010*"weather" + 0.008*"queensland" + 0.008*"farm" + 0.007*"flood" + 0.006*"friday" + 0.006*"farmer" + 0.006*"damag" + 0.006*"rain" + 0.006*"dairi" + 0.006*"cyclon"

Score: 0.30491358041763306	 
Topic: 0.017*"countri" + 0.017*"hour" + 0.014*"rural" + 0.013*"market" + 0.011*"news" + 0.008*"share" + 0.008*"price" + 0.007*"busi" + 0.006*"rise" + 0.006*"nation"

Score: 0.020002121105790138	 
Topic: 0.022*"trump" + 0.007*"novemb" + 0.007*"monday" + 0.006*"wednesday" + 0.005*"decemb" + 0.005*"leagu" + 0.005*"open" + 0.005*"world" + 0.004*"final" + 0.004*"round"

Score: 0.020002055913209915	 
Topic: 0.012*"govern" + 0.010*"elect" + 0.007*"drum" + 0.007*"labor" + 0.007*"health" + 0.007*"turnbul" + 0.007*"fund" + 0.006*"say" + 0.006*"budget" + 0.005*"feder"

Score: 0.0200016051530838	 
Topic: 0.016*"polic" + 0.014*"interview" + 0.013*"charg" + 0.013*"crash" + 0.013*"murder" + 0.012*"woman" + 0.010*"death" + 0.009*"court" + 0.008*"shoot" + 0.008*"die"

Score:

In [29]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'

bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.2811703383922577	 Topic: 0.029*"elect" + 0.019*"say" + 0.018*"death" + 0.017*"hospit" + 0.016*"tasmanian"
Score: 0.2521570026874542	 Topic: 0.020*"nation" + 0.018*"coast" + 0.016*"help" + 0.016*"countri" + 0.015*"state"
Score: 0.18333333730697632	 Topic: 0.019*"canberra" + 0.018*"market" + 0.014*"rise" + 0.014*"west" + 0.014*"australian"
Score: 0.18333332240581512	 Topic: 0.062*"polic" + 0.023*"crash" + 0.019*"interview" + 0.018*"miss" + 0.018*"shoot"
Score: 0.016672654077410698	 Topic: 0.036*"trump" + 0.032*"australian" + 0.019*"queensland" + 0.014*"leav" + 0.013*"australia"
Score: 0.01666666567325592	 Topic: 0.029*"charg" + 0.027*"court" + 0.021*"murder" + 0.018*"woman" + 0.018*"face"
Score: 0.01666666567325592	 Topic: 0.035*"australia" + 0.021*"melbourn" + 0.021*"world" + 0.017*"open" + 0.014*"final"
Score: 0.01666666567325592	 Topic: 0.026*"south" + 0.025*"kill" + 0.015*"island" + 0.013*"fall" + 0.011*"attack"
Score: 0.01666666567325592	 Topic: 0.020*"council" + 0.015*"pow